In [3]:
pip install pyperclip

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyperclip: filename=pyperclip-1.9.0-py3-none-any.whl size=11019 sha256=9579d2c3b3dacd4d8ee5ce384273790f2eb372e8e2f0436299b29a5bf95635e2
  Stored in directory: c:\users\tanvi\appdata\local\pip\cache\wheels\e0\e8\fc\8ab8aa326e33bc066ccd5f3ca9646eab4299881af933f94f09
Successfully built pyperclip
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install googletrans==4.0.0-rc1 pykakasi pypinyin hangul-romanize pyperclip

Note: you may need to restart the kernel to use updated packages.


In [20]:
from tkinter import *
from tkinter import ttk, messagebox
from googletrans import Translator, LANGUAGES
import pykakasi
from pypinyin import lazy_pinyin
from hangul_romanize import Transliter
from hangul_romanize.rule import academic
import pyperclip

translator = Translator()
kks = pykakasi.kakasi()
transliter = Transliter(academic)

def get_lang_name(lang_code):
    """Convert language code to full language name (e.g., 'ja' -> 'Japanese')"""
    return LANGUAGES.get(lang_code, lang_code).capitalize()

def romanize_text(text, lang_code):
    try:
        if lang_code == 'ja':  # Japanese
            result = kks.convert(text)
            return ' '.join([item['hepburn'] for item in result])
        elif lang_code in ['zh-cn', 'zh-tw']:  # Chinese
            return ' '.join(lazy_pinyin(text))
        elif lang_code == 'ko':  # Korean
            return transliter.translit(text)
        else:
            return None
    except Exception:
        return None

def translate_text():
    input_text = text_input.get("1.0", END).strip()
    selected_lang = lang_choice.get()
    lang_code_list = [k for k, v in LANGUAGES.items() if v == selected_lang.lower()]

    if not input_text:
        messagebox.showwarning("Warning", "Please enter some text.")
        return

    if not lang_code_list:
        messagebox.showerror("Error", "Selected language is not supported.")
        return

    target_lang_code = lang_code_list[0]

    try:
        # Detect source language
        detected = translator.detect(input_text)
        source_lang_code = detected.lang
        source_lang_name = get_lang_name(source_lang_code)

        # Update label to show detected language
        source_lang_label.config(text=f"Detected Source Language: {source_lang_name}")

        # Translate
        translated = translator.translate(input_text, src=source_lang_code, dest=target_lang_code)

        text_output.config(state="normal")
        text_output.delete("1.0", END)
        text_output.insert(END, translated.text)

        # Add pronunciation if supported
        romanized = romanize_text(translated.text, target_lang_code)
        if romanized:
            text_output.insert(END, f"\n\nPronunciation: {romanized}")

        text_output.config(state="disabled")

    except Exception as e:
        messagebox.showerror("Translation Error", f"An error occurred: {e}")

def clear_text():
    text_input.delete("1.0", END)
    text_output.config(state="normal")
    text_output.delete("1.0", END)
    text_output.config(state="disabled")
    source_lang_label.config(text="Detected Source Language:")  # Reset label

def copy_output():
    output = text_output.get("1.0", END).strip()
    if output:
        pyperclip.copy(output)
        messagebox.showinfo("Copied", "Translated text copied to clipboard.")

# GUI Setup
root = Tk()
root.title("Real-Time Translator with Pronunciation")
root.geometry("650x520")
root.configure(bg="#6caccf")

style = ttk.Style()
style.theme_use("clam")

frame = Frame(root, bg="#6caccf", padx=20, pady=20)
frame.pack(fill=BOTH, expand=True)

# Input area
Label(frame, text="Enter Text", font=("Helvetica", 12, "bold"), bg="#6caccf").pack(anchor=W)
text_input = Text(frame, height=5, font=("Arial", 11), bg="#eff0f1")
text_input.pack(fill=X, pady=(5, 10))

# Language selector
Label(frame, text="Select Language", font=("Helvetica", 12, "bold"), bg="#6caccf").pack(anchor=W)
lang_choice = StringVar(root)
lang_choice.set("japanese")
lang_menu = ttk.Combobox(frame, textvariable=lang_choice, values=sorted(list(LANGUAGES.values())), font=("Arial", 11))
lang_menu.pack(fill=X, pady=(5, 15))

# Buttons
button_frame = Frame(frame, bg="#6caccf")
button_frame.pack(fill=X)

ttk.Button(button_frame, text="Translate", command=translate_text).pack(side=LEFT, padx=(0, 10))
ttk.Button(button_frame, text="Clear", command=clear_text).pack(side=LEFT, padx=(0, 10))
ttk.Button(button_frame, text="Copy", command=copy_output).pack(side=LEFT)

# Detected language label
source_lang_label = Label(frame, text="Detected Source Language:", font=("Helvetica", 11, "italic"), bg="#6caccf")
source_lang_label.pack(anchor=W, pady=(15, 5))

# Output area
Label(frame, text="Translated Text", font=("Helvetica", 12, "bold"), bg="#6caccf").pack(anchor=W, pady=(0, 5))
text_output = Text(frame, height=10, font=("Arial", 11), bg="#eff0f1")
text_output.pack(fill=X)
text_output.config(state="disabled")  # Initially read-only

root.mainloop()
